In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Arrhythmia/GSE143924'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptome analysis from human epicardial adipose tissue biopsies analyzed according to postoperative atrial fibrillation occurrence after cardiac surgery"
!Series_summary	"Introduction: Post-operative atrial fibrillation (POAF) is a frequent complication after cardiac surgery, but its pathophysiology remains incompletely understood. Considering that epicardial adipose tissue (EAT) is in close vicinity with the atrial myocardium, we hypothesized that a specific pre-operative EAT phenotype would be associated to POAF onset following surgery.  Methods: Patients undergoing cardiac surgery prospectively enrolled in the POMI-AF cohort between February 2016 and June 2017 were studied. EAT samples were collected at the beginning of surgery. Whole-tissue gene expression patterns and the stromal and vascular fraction (SVF) cellular composition were explored. Patients were followed after surgery by continuous ECG to detect POAF onset.  Results: Among the

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 1  # based on given sample characteristics
age_row = None  # age information not present
gender_row = None  # gender information not present

# Functions for data type conversion
def convert_trait(value):
    if "postoperative atrial fibrillation after surgery (POAF)" in value:
        return 1
    elif "sinus rhythm after surgery" in value:
        return 0
    else:
        return None

convert_age = convert_gender = None  # Not applicable as data is not available

save_cohort_info('GSE143924', './preprocessed/Arrhythmia/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Arrhythmia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Arrhythmia/trait_data/GSE143924.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4276706': [0], 'GSM4276707': [0], 'GSM4276708': [0], 'GSM4276709': [0], 'GSM4276710': [0], 'GSM4276711': [0], 'GSM4276712': [0], 'GSM4276713': [0], 'GSM4276714': [0], 'GSM4276715': [0], 'GSM4276716': [0], 'GSM4276717': [0], 'GSM4276718': [0], 'GSM4276719': [0], 'GSM4276720': [0], 'GSM4276721': [1], 'GSM4276722': [1], 'GSM4276723': [1], 'GSM4276724': [1], 'GSM4276725': [1], 'GSM4276726': [1], 'GSM4276727': [1], 'GSM4276728': [1], 'GSM4276729': [1], 'GSM4276730': [1], 'GSM4276731': [1], 'GSM4276732': [1], 'GSM4276733': [1], 'GSM4276734': [1], 'GSM4276735': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])
    

# Based on domain knowledge and common gene naming conventions, these IDs are human gene symbols.


Index(['A2M-AS1', 'AACS', 'AADAC', 'AADACL2', 'AADACP1', 'AAK1', 'AARD',
       'AASS', 'ABCA1', 'ABCA10', 'ABCA11P', 'ABCA3', 'ABCA5', 'ABCA9-AS1',
       'ABCB1', 'ABCB11', 'ABCB4', 'ABCB7', 'ABCC3', 'ABCC6'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# No gene mapping required in this case. Hence, skipping STEP6. 


Gene annotation preview:
{'ID': ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1'], 'SPOT_ID': ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1']}


### Step 7: Data Normalization and Merging

In [7]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Arrhythmia/gene_data/GSE143924.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Arrhythmia')

# 4. Save the cohort information.
save_cohort_info('GSE143924', './preprocessed/Arrhythmia/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Arrhythmia/GSE143924.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Arrhythmia', the least common label is '0.0' with 15 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Arrhythmia' in this dataset is fine.

